In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')
from scipy.io import wavfile
import scipy.signal
from PIL import Image
import shutil

In [ ]:
#import noisereduce as nr
from imblearn.over_sampling import SMOTE
from scipy.signal import lfilter, butter
#import keras_tuner as kt
#from kerastuner import HyperModel
#from keras_tuner import RandomSearch

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
#from hmmlearn import hmm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  VotingClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from scipy.stats import uniform, randint
from keras import optimizers
from keras import losses


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM,GRU
from keras import regularizers
from sklearn.neural_network import MLPClassifier
from keras.optimizers import Adam, SGD, RMSprop, Adadelta
from tensorflow.keras import layers, models
from tensorflow.keras.layers import SimpleRNN
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.layers import  Bidirectional,RepeatVector, TimeDistributed
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

from keras.layers import BatchNormalization

In [ ]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
100% 428M/429M [00:03<00:00, 122MB/s]
100% 429M/429M [00:03<00:00, 132MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/ravdess-emotional-speech-audio.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
#filename: ex-03-01-06-01-02-01-12.wav ,where third therm is Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised)
paths = []
labels = []
for dirname, _, filenames in os.walk('/content/audio_speech_actors_01-24'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        parts = filename.split('-')

        if len(parts) >= 3:
            label = parts[2]  # Get the 3rd term
            labels.append(label)
        else:
            print(f"Skipping file with unexpected format: {filename}")


In [ ]:
## Create a RAVDESS dataframe
df1 = pd.DataFrame()
df1['speech'] = paths
df1['label'] = labels
df1.head()

,speech,label
0,/content/audio_speech_actors_01-24/Actor_05/03...,05
1,/content/audio_speech_actors_01-24/Actor_05/03...,07
2,/content/audio_speech_actors_01-24/Actor_05/03...,02
3,/content/audio_speech_actors_01-24/Actor_05/03...,04
4,/content/audio_speech_actors_01-24/Actor_05/03...,02


In [ ]:

label_count = df1['label'].value_counts()
label_count

,count
label,
05,192
07,192
02,192
04,192
06,192
08,192
03,192
01,96


### Feature extraction

In [ ]:
!pip install antropy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.8 MB/s eta 0:00:00
  Created wheel for antropy: filename=antropy-0.1.6-py3-none-any.whl size=16879 sha256=c1d3254375978e46fbacba6c31b4014fdb5931c55a47d1ad229dcf82733b72c6
  Stored in directory: /root/.cache/pip/wheels/98/22/06/e91d7bb213c7133d5e2eb34258623e1e19928d5f05e1ee6812
Successfully built antropy


In [ ]:
from scipy.stats import entropy
import antropy as ant

In [ ]:
# Define preprocessing parameters
TARGET_SAMPLE_RATE = 48000   # Desired sample rate for resampling
FRAME_SIZE = 1200            # Frame size in samples (25 ms)
HOP_LENGTH = 600             # Hop length in samples (50% overlap)
MFCC_N_MFCC = 13            # Number of MFCC coefficients to extract
PAD_LENGTH = 5 * TARGET_SAMPLE_RATE

In [ ]:
# Function to preprocess audio
def preprocess_audio(file_path, target_sr=TARGET_SAMPLE_RATE, pad_length=PAD_LENGTH):
    # Load the audio
    y, sr = librosa.load(file_path, sr=target_sr)

    # Step 1: Silence Removal (Using librosa.effects.trim)
    y, _ = librosa.effects.trim(y)

    # Step 2: Padding (if the audio is shorter than the target length)
    if len(y) < pad_length:
        padding = pad_length - len(y)
        y = np.pad(y, (0, padding), 'constant')  # Zero padding at the end

    # Step 3: Normalization (Peak normalization to [-1, 1] range)
    y = librosa.util.normalize(y)

    return y, sr

In [ ]:
# Function to compute spectral entropy based on MFCC
def spectral_entropy(mfcc):
    # Normalize the MFCC to form a probability distribution
    psd = np.square(mfcc)
    psd_norm = psd / np.sum(psd, axis=0)

    # Compute the spectral entropy (Shannon entropy)
    entropy = -np.sum(psd_norm * np.log(psd_norm + 1e-12), axis=0)
    entropy_mean = entropy.mean()  # Mean spectral entropy over time
    return entropy_mean


In [ ]:
# Function to compute approximate entropy
def approximate_entropy(mfcc, m=2, r=0.2):
    def _phi(m):
        N = mfcc.shape[1] - m + 1
        patterns = np.array([mfcc[:, i:i+m] for i in range(N)])
        distances = np.sum(np.abs(patterns[:, np.newaxis] - patterns[np.newaxis, :]), axis=-1)
        Cm = np.sum(np.max(distances, axis=2) <= r, axis=1) / N
        return np.sum(np.log(Cm + 1e-12)) / N

    # Approximate entropy calculation
    return abs(_phi(m+1) - _phi(m))

In [ ]:
# Function to extract MFCC mean, spectral entropy, and approximate entropy
def extract_features(audio, sr, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH, n_mfcc=MFCC_N_MFCC):
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)

    # Compute the mean of each MFCC coefficient over time
    mfcc_means = mfcc.mean(axis=1)

    # Compute spectral entropy using MFCC
    spec_entropy = spectral_entropy(mfcc)

    # Compute approximate entropy using MFCC
    approx_entropy = approximate_entropy(mfcc)

    return mfcc_means, spec_entropy, approx_entropy

In [ ]:
# Initialize list to store processed data
processed_data = []

# Assuming df1 is already defined and contains the audio paths and labels
for index, row in df1.iterrows():
    audio_path = row['speech']
    label = row['label']

    # Preprocess the audio
    y, sr = preprocess_audio(audio_path)

    # Extract features
    mfcc_means, spec_entropy, approx_entropy = extract_features(y, sr)

    # Combine all features into a single array
    features = np.hstack([mfcc_means, spec_entropy, approx_entropy])

    # Append the features and label to processed_data
    processed_data.append((features, label))

In [ ]:
# Convert processed_data to DataFrame
features_array, labels = zip(*processed_data)  # Unzip features and labels
features_df = pd.DataFrame(features_array)  # Create DataFrame for features
features_df['label'] = labels  # Add labels as a new column

In [ ]:
df=features_df
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
0,-482.506104,59.905594,-6.821089,2.734158,1.233579,8.958209,-11.159545,-2.004997,-1.493389,-2.094221,-4.677904,-2.987441,0.164259,0.346328,0.030019,05
1,-434.612213,50.399967,0.631297,16.207701,2.889047,6.601787,-5.060184,2.132876,0.980047,1.502715,1.947337,1.624204,4.511497,0.307142,0.052273,07
2,-422.564453,66.020157,9.411767,19.945904,10.494566,7.528195,1.195160,3.965173,1.304804,0.231898,4.683016,1.790579,1.876973,0.350052,0.008778,02
3,-441.973785,45.318970,8.326529,12.036758,5.382025,9.978813,0.852225,4.016699,-0.788007,-2.175442,-0.144637,3.107192,2.169956,0.282367,0.022878,04
4,-429.680267,63.371269,11.470847,25.317907,10.043150,10.997593,0.675366,4.491855,0.114406,-2.783764,5.472361,4.758094,3.971247,0.357398,0.008628,02


In [ ]:
x=df.drop(columns=['label'])

In [ ]:
x.shape

(1440, 15)

In [ ]:
y=df['label']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)


In [ ]:
type(x)

numpy.ndarray

In [ ]:
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [ ]:
y = y.toarray()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
y_train_sparse = tf.argmax(y_train, axis=1)
y_test_sparse = tf.argmax(y_test, axis=1)

In [ ]:
# Initialize the Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42)

# Fit the model on the training data
dt_model.fit(x_train, y_train_sparse)


DecisionTreeClassifier(random_state=42)

In [ ]:
# Make predictions on the test data
y_pred = dt_model.predict(x_test)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test_sparse, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test_sparse, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test_sparse, y_pred))

Accuracy: 0.32
Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.22      0.17        18
           1       0.39      0.44      0.42        34
           2       0.30      0.29      0.29        38
           3       0.27      0.23      0.25        44
           4       0.39      0.28      0.32        40
           5       0.28      0.30      0.29        33
           6       0.42      0.32      0.36        44
           7       0.33      0.43      0.37        37

    accuracy                           0.32       288
   macro avg       0.31      0.31      0.31       288
weighted avg       0.33      0.32      0.32       288

Confusion Matrix:
[[ 4  3  1  2  2  2  3  1]
 [ 6 15  1  6  0  1  1  4]
 [ 1  2 11  7  1  6  3  7]
 [ 8  7  5 10  2  2  6  4]
 [ 4  0  2  4 11  8  5  6]
 [ 3  2  7  2  5 10  1  3]
 [ 1  6  4  4  4  3 14  8]
 [ 3  3  6  2  3  4  0 16]]


In [ ]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_model.fit(x_train, y_train_sparse)

# Make predictions on the test data
y_pred = rf_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test_sparse, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.54


In [ ]:
# Initialize the LightGBM Classifier
lgbm_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
lgbm_model.fit(x_train, y_train_sparse)

# Make predictions on the test data
y_pred = lgbm_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test_sparse, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test_sparse, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test_sparse, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000929 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 1152, number of used features: 15
[LightGBM] [Info] Start training from score -2.692546
[LightGBM] [Info] Start training from score -1.986660
[LightGBM] [Info] Start training from score -2.012302
[LightGBM] [Info] Start training from score -2.052043
[LightGBM] [Info] Start training from score -2.025374
[LightGBM] [Info] Start training from score -1.980351
[LightGBM] [Info] Start training from score -2.052043
[LightGBM] [Info] Start training from score -2.005830
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.52
Classification Report:
              pre

In [ ]:
# Initialize the Support Vector Classifier
svm_model = SVC(kernel='poly', random_state=42)

In [ ]:
# Fit the model on the training data
svm_model.fit(x_train, y_train_sparse)

SVC(kernel='poly', random_state=42)

In [ ]:
# Make predictions on the test data
y_pred = svm_model.predict(x_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test_sparse, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test_sparse, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test_sparse, y_pred))

Accuracy: 0.40
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.06      0.11        18
           1       0.50      0.41      0.45        34
           2       0.52      0.42      0.46        38
           3       0.30      0.18      0.23        44
           4       0.65      0.33      0.43        40
           5       0.52      0.33      0.41        33
           6       0.62      0.45      0.53        44
           7       0.25      0.86      0.39        37

    accuracy                           0.40       288
   macro avg       0.55      0.38      0.38       288
weighted avg       0.51      0.40      0.39       288

Confusion Matrix:
[[ 1  2  1  1  0  1  0 12]
 [ 0 14  1  5  0  0  4 10]
 [ 0  1 16  6  0  3  0 12]
 [ 0  6  2  8  3  1  2 22]
 [ 0  3  4  2 13  1  4 13]
 [ 0  1  3  3  2 11  0 13]
 [ 0  1  1  2  2  4 20 14]
 [ 0  0  3  0  0  0  2 32]]


# **MLP models**

In [ ]:
#Multi Layer Perceptropn
# Create MLP model
model = Sequential()


model.add(Dense(52, input_dim=13, activation='elu'))  # Adjust input shape according to your dataset
model.add(BatchNormalization())
model.add(Dropout(0.1))


model.add(Dense(130,  activation='elu'))  # Adjust input shape according to your dataset
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(130, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(130, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(52, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history=model.fit(x_train, y_train_sparse, batch_size=50, epochs=100, validation_data=(x_test, y_test_sparse))

Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 13, but received input with shape (None, 15)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 15), dtype=float32)
  • training=True
  • mask=None

## MLP parameter tunning


In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt
from kerastuner import HyperModel
from keras_tuner import RandomSearch

### 1st

In [ ]:
# Define the HyperModel class:1
class MLPHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 8)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=52, max_value=390, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=64, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

Trial 3 Complete [00h 00m 34s]
val_accuracy: 0.6805555820465088

Best val_accuracy So Far: 0.6805555820465088
Total elapsed time: 00h 01m 21s


In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")

Best Hyperparameters:
num_layers: 3
units_0: 364
activation_0: relu
dropout_0: 0.4
learning_rate: 0.006019793254027166
loss: sparse_categorical_crossentropy
units_1: 377
activation_1: elu
dropout_1: 0.1
units_2: 91
activation_2: relu
dropout_2: 0.1
units_3: 221
activation_3: relu
dropout_3: 0.5
units_4: 286
activation_4: tanh
dropout_4: 0.5
units_5: 286
activation_5: softmax
dropout_5: 0.1
units_6: 156
activation_6: elu
dropout_6: 0.4


In [ ]:
    model = Sequential()

    # Layer 0
    model.add(Dense(364, activation='relu', input_shape=(13,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    # Layer 1
    model.add(Dense(377, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    # Layer 2
    model.add(Dense(91, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    # Layer 3
    model.add(Dense(221, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Layer 4
    model.add(Dense(286, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Layer 5
    model.add(Dense(286, activation='softmax'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    # Layer 6 (Output layer)
    model.add(Dense(156, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.006019793254027166)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.0189 - loss: 15.0847 - val_accuracy: 0.1319 - val_loss: 2.1610
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0933 - loss: 12.1662 - val_accuracy: 0.1319 - val_loss: 2.2524
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1262 - loss: 10.6082 - val_accuracy: 0.1562 - val_loss: 2.2592
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0857 - loss: 10.1090 - val_accuracy: 0.1597 - val_loss: 8.0742
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0916 - loss: 8.9078 - val_accuracy: 0.1736 - val_loss: 4.5027
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1129 - loss: 7.6706 - val_accuracy: 0.1354 - val_loss: 10.6071
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1225 - loss: 7.0562 - val_accuracy: 0.1458 - val_loss: 6.6042
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1011 - loss: 6.3886 - val_accuracy: 0.1

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.1689 - loss: 38.6983 - val_accuracy: 0.2604 - val_loss: 2.0617
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1900 - loss: 2.5422 - val_accuracy: 0.1354 - val_loss: 2.2483
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1550 - loss: 2.9964 - val_accuracy: 0.1354 - val_loss: 2.3363
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1378 - loss: 2.6204 - val_accuracy: 0.1562 - val_loss: 2.2311
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1116 - loss: 2.7879 - val_accuracy: 0.1562 - val_loss: 2.4742
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1443 - loss: 2.6668 - val_accuracy: 0.1528 - val_loss: 2.2475
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1455 - loss: 3.9503 - val_accuracy: 0.1597 - val_loss: 2.6152
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1123 - loss: 2.7014 - val_accuracy: 0.1562 -

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='relu',input_dim=13))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.1777 - loss: 3.1252 - val_accuracy: 0.1701 - val_loss: 2.0325
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2198 - loss: 2.0233 - val_accuracy: 0.2222 - val_loss: 1.9109
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2681 - loss: 1.8743 - val_accuracy: 0.3299 - val_loss: 1.7557
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3024 - loss: 1.7826 - val_accuracy: 0.3542 - val_loss: 1.7332
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3726 - loss: 1.7496 - val_accuracy: 0.3993 - val_loss: 1.6415
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3943 - loss: 1.6275 - val_accuracy: 0.4271 - val_loss: 1.6411
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3786 - loss: 1.6350 - val_accuracy: 0.3819 - val_loss: 1.8500
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4057 - loss: 1.5563 - val_accuracy: 0.4062 - 

###67

In [ ]:
model=Sequential()
model.add(Dense(390,activation='relu',input_dim=15))
model.add(Dense(780,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.2108 - loss: 3.1166 - val_accuracy: 0.2396 - val_loss: 3.4816
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3168 - loss: 2.0539 - val_accuracy: 0.2257 - val_loss: 5.2246
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.3328 - loss: 1.8741 - val_accuracy: 0.3333 - val_loss: 5.3599
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4145 - loss: 1.6940 - val_accuracy: 0.3472 - val_loss: 3.4947
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4418 - loss: 1.5487 - val_accuracy: 0.3819 - val_loss: 2.0382
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5241 - loss: 1.4615 - val_accuracy: 0.4653 - val_loss: 1.8102
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5152 - loss: 1.4020 - val_accuracy: 0.4410 - val_loss: 1.9584
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5658 - loss: 1.2238 - val_accuracy: 0.

### 68

In [ ]:
model=Sequential()
model.add(Dense(390,activation='relu',input_dim=15))
model.add(Dense(780,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=100,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.1675 - loss: 2.7843 - val_accuracy: 0.1389 - val_loss: 2.0588
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.2841 - loss: 2.2466 - val_accuracy: 0.1389 - val_loss: 2.0556
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.3704 - loss: 1.8275 - val_accuracy: 0.1389 - val_loss: 2.0566
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.4060 - loss: 1.7211 - val_accuracy: 0.1493 - val_loss: 2.0535
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.4656 - loss: 1.5619 - val_accuracy: 0.1597 - val_loss: 2.0316
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.4826 - loss: 1.4961 - val_accuracy: 0.2118 - val_loss: 1.9779
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5308 - loss: 1.2902 - val_accuracy: 0.2708 - val_loss: 1.8928
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.5546 - loss: 1.2572 - val_accuracy: 0.

### 72

In [ ]:
model=Sequential()
model.add(Dense(390,activation='relu',input_dim=15))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(390,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=64, validation_data=(x_test, y_test_sparse))

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.1310 - loss: 2.9904 - val_accuracy: 0.1875 - val_loss: 2.0596
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.2628 - loss: 2.2985 - val_accuracy: 0.2118 - val_loss: 2.0518
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.2890 - loss: 2.0390 - val_accuracy: 0.2049 - val_loss: 2.0465
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3471 - loss: 1.9379 - val_accuracy: 0.2083 - val_loss: 2.0377
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.3990 - loss: 1.7825 - val_accuracy: 0.2118 - val_loss: 2.0275
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4417 - loss: 1.6255 - val_accuracy: 0.2188 - val_loss: 2.0163
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4492 - loss: 1.5924 - val_accuracy: 0.2153 - val_loss: 1.9924
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4825 - loss: 1.4341 - val_accuracy: 0.

In [ ]:
model=Sequential()
model.add(Dense(390,activation='relu',input_dim=13))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(390,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=64, validation_data=(x_test, y_test_sparse))

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.1478 - loss: 3.0487 - val_accuracy: 0.2014 - val_loss: 2.0523
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1751 - loss: 2.7647 - val_accuracy: 0.2083 - val_loss: 2.0393
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2294 - loss: 2.3136 - val_accuracy: 0.2153 - val_loss: 2.0289
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2654 - loss: 2.2212 - val_accuracy: 0.2361 - val_loss: 2.0165
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2792 - loss: 2.1893 - val_accuracy: 0.2465 - val_loss: 2.0078
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2827 - loss: 2.1120 - val_accuracy: 0.1979 - val_loss: 1.9979
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3271 - loss: 1.9307 - val_accuracy: 0.2083 - val_loss: 1.9873
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3825 - loss: 1.9076 - val_accuracy: 0.2188 -

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='elu',input_dim=13))
model.add(Dense(1024,activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# Compile the model
optimizer = RMSprop(learning_rate=0.006019793254027166)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

###2nd

In [ ]:
# Define the HyperModel class
class MLPHyperModel2(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 110)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=52, max_value=500, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax','sigmoid']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.8, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel2(),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning1'
)

In [ ]:
tuner.search(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

Trial 2 Complete [00h 00m 15s]

Best val_accuracy So Far: None
Total elapsed time: 00h 01m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
20                |102               |num_layers
390               |351               |units_0
softmax           |sigmoid           |activation_0
0.5               |0.5               |dropout_0
0.00060174        |0.073653          |learning_rate
categorical_cro...|categorical_cro...|loss
494               |52                |units_1
tanh              |relu              |activation_1
0.5               |0.1               |dropout_1
416               |52                |units_2
sigmoid           |relu              |activation_2
0.8               |0.1               |dropout_2
247               |52                |units_3
tanh              |relu              |activation_3
0.1               |0.1               |dropout_3
221               |52                |units_4
relu              |relu              |activation_4
0.7 

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/py

In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")

Best Hyperparameters:
num_layers: 5
units_0: 65
activation_0: relu
dropout_0: 0.1
learning_rate: 0.0010343990168720423
loss: sparse_categorical_crossentropy
units_1: 260
activation_1: relu
dropout_1: 0.30000000000000004
units_2: 364
activation_2: relu
dropout_2: 0.4
units_3: 52
activation_3: relu
dropout_3: 0.1
units_4: 52
activation_4: relu
dropout_4: 0.1


In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 52, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape=(13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers
for i in range(num_layers):
    model.add(layers.Dense(units[i], activation=activations[i]))
    model.add(layers.Dropout(dropouts[i]))

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 8)                   │             424 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 135,234 (528.26 KB)

 Trainable params: 135,234 (528.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.1262 - loss: 2.0705 - val_accuracy: 0.2396 - val_loss: 1.9188
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2204 - loss: 1.9371 - val_accuracy: 0.3021 - val_loss: 1.8407
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3010 - loss: 1.8134 - val_accuracy: 0.3646 - val_loss: 1.7210
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3317 - loss: 1.7524 - val_accuracy: 0.3507 - val_loss: 1.6648
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3507 - loss: 1.7107 - val_accuracy: 0.4132 - val_loss: 1.5675
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4198 - loss: 1.5629 - val_accuracy: 0.4132 - val_loss: 1.5636
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3980 - loss: 1.5792 - val_accuracy: 0.4340 - val_loss: 1.4833
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4264 - loss: 1.5012 - val_accuracy: 0.4653 - 

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [364, 260, 364, 260, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape=(13,)


In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers
for i in range(num_layers):
    model.add(layers.Dense(units[i], activation=activations[i]))
    model.add(layers.Dropout(dropouts[i]))

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                     │ (None, 364)                 │           5,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 260)                 │          94,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 260)                 │          94,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 52)                  │          13,572 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 8)                   │             424 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 303,896 (1.16 MB)

 Trainable params: 303,896 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.1656 - loss: 2.0143 - val_accuracy: 0.3368 - val_loss: 1.7937
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2860 - loss: 1.8165 - val_accuracy: 0.3646 - val_loss: 1.6659
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3856 - loss: 1.6549 - val_accuracy: 0.4028 - val_loss: 1.5180
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3876 - loss: 1.5604 - val_accuracy: 0.4236 - val_loss: 1.4685
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4637 - loss: 1.4459 - val_accuracy: 0.4618 - val_loss: 1.4761
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4694 - loss: 1.3766 - val_accuracy: 0.4653 - val_loss: 1.3858
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5304 - loss: 1.2715 - val_accuracy: 0.4826 - val_loss: 1.4469
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5402 - loss: 1.2479 - val_accuracy: 0.479

In [ ]:
# Define the MLP model parameters
num_layers = 6
units = [65, 260, 364, 52, 52,26]
activations = ['relu', 'relu', 'relu', 'relu', 'relu','relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1,0.5]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape=(13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers
for i in range(num_layers):
    model.add(layers.Dense(units[i], activation=activations[i]))
    model.add(layers.Dropout(dropouts[i]))

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 26)                  │           1,378 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 26)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 8)                   │             216 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136,404 (532.83 KB)

 Trainable params: 136,404 (532.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.1260 - loss: 2.0862 - val_accuracy: 0.2222 - val_loss: 2.0089
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2054 - loss: 2.0185 - val_accuracy: 0.3333 - val_loss: 1.9350
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2043 - loss: 1.9862 - val_accuracy: 0.3090 - val_loss: 1.8928
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2339 - loss: 1.9377 - val_accuracy: 0.3403 - val_loss: 1.8407
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2571 - loss: 1.9415 - val_accuracy: 0.3611 - val_loss: 1.8076
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2469 - loss: 1.8711 - val_accuracy: 0.3438 - val_loss: 1.6984
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2791 - loss: 1.8499 - val_accuracy: 0.3507 - val_loss: 1.6802
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3113 - loss: 1.7885 - val_accuracy: 0.4028 - 

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 52, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape = (13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization
for i in range(num_layers):
    model.add(layers.Dense(units[i]))
    model.add(layers.BatchNormalization())  # Add batch normalization
    model.add(layers.Activation(activations[i]))  # Apply activation after batch normalization
    model.add(layers.Dropout(dropouts[i]))  # Add dropout for regularization

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 52)                  │              

 Total params: 138,406 (540.65 KB)

 Trainable params: 136,820 (534.45 KB)

 Non-trainable params: 1,586 (6.20 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.1416 - loss: 2.3471 - val_accuracy: 0.2604 - val_loss: 1.9943
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2369 - loss: 1.9926 - val_accuracy: 0.3160 - val_loss: 1.9399
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2904 - loss: 1.8482 - val_accuracy: 0.3472 - val_loss: 1.8477
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3435 - loss: 1.8061 - val_accuracy: 0.3472 - val_loss: 1.7936
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3339 - loss: 1.7242 - val_accuracy: 0.3958 - val_loss: 1.7294
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3592 - loss: 1.6830 - val_accuracy: 0.4340 - val_loss: 1.6619
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3588 - loss: 1.6768 - val_accuracy: 0.4236 - val_loss: 1.6048
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3647 - loss: 1.6591 - val_accuracy: 0.4375 -

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 52, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape = (13,)

###3rd


In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization
for i in range(num_layers):
    model.add(layers.Dense(units[i]))
    model.add(layers.BatchNormalization())  # Add batch normalization
    model.add(layers.Activation(activations[i]))  # Apply activation after batch normalization
    model.add(layers.Dropout(dropouts[i]))  # Add dropout for regularization

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_70 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_42               │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_30 (Activation)           │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_59 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_43               │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_31 (Activation)           │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_60 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_72 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_44               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_32 (Activation)           │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_61 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_45               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_33 (Activation)           │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_62 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_46               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_34 (Activation)           │ (None, 52)                  │              

 Total params: 138,406 (540.65 KB)

 Trainable params: 136,820 (534.45 KB)

 Non-trainable params: 1,586 (6.20 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=184,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.1476 - loss: 2.2927 - val_accuracy: 0.2674 - val_loss: 1.9855
Epoch 2/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2413 - loss: 1.9276 - val_accuracy: 0.2917 - val_loss: 1.9125
Epoch 3/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3031 - loss: 1.7990 - val_accuracy: 0.3368 - val_loss: 1.8301
Epoch 4/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3752 - loss: 1.6884 - val_accuracy: 0.3542 - val_loss: 1.7429
Epoch 5/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3674 - loss: 1.6580 - val_accuracy: 0.3854 - val_loss: 1.6698
Epoch 6/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3893 - loss: 1.6570 - val_accuracy: 0.4375 - val_loss: 1.5930
Epoch 7/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4174 - loss: 1.5846 - val_accuracy: 0.4722 - val_loss: 1.5286
Epoch 8/184
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4141 - loss: 1.5328 - val_accuracy: 0.4896 - 

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 364, 364, 260, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape = (13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization
for i in range(num_layers):
    model.add(layers.Dense(units[i]))
    model.add(layers.BatchNormalization())  # Add batch normalization
    model.add(layers.Activation(activations[i]))  # Apply activation after batch normalization
    model.add(layers.Dropout(dropouts[i]))  # Add dropout for regularization

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 364)                 │          24,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 364)                 │         132,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 260)                 │          94,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 52)                  │          13,572 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 52)                  │              

 Total params: 271,110 (1.03 MB)

 Trainable params: 268,900 (1.03 MB)

 Non-trainable params: 2,210 (8.63 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.1711 - loss: 2.3047 - val_accuracy: 0.3403 - val_loss: 1.9131
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2703 - loss: 1.9560 - val_accuracy: 0.2917 - val_loss: 1.8520
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3700 - loss: 1.7096 - val_accuracy: 0.3194 - val_loss: 1.7919
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3686 - loss: 1.6573 - val_accuracy: 0.3889 - val_loss: 1.6897
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3905 - loss: 1.6165 - val_accuracy: 0.4132 - val_loss: 1.5885
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4466 - loss: 1.5268 - val_accuracy: 0.4340 - val_loss: 1.5336
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4437 - loss: 1.5160 - val_accuracy: 0.4583 - val_loss: 1.4867
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4515 - loss: 1.4650 - val_accuracy: 0.4306 -

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 52, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape = (13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization
for i in range(num_layers):
    model.add(layers.Dense(units[i]))
    model.add(layers.BatchNormalization())  # Add batch normalization
    model.add(layers.Activation(activations[i]))  # Apply activation after batch normalization
    model.add(layers.Dropout(dropouts[i]))  # Add dropout for regularization

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_8 (Activation)            │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_9 (Activation)            │ (None, 52)                  │              

 Total params: 138,406 (540.65 KB)

 Trainable params: 136,820 (534.45 KB)

 Non-trainable params: 1,586 (6.20 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.1567 - loss: 2.2879 - val_accuracy: 0.2812 - val_loss: 1.9944
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2290 - loss: 1.9659 - val_accuracy: 0.2743 - val_loss: 1.9487
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2866 - loss: 1.8746 - val_accuracy: 0.3368 - val_loss: 1.8808
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2984 - loss: 1.8286 - val_accuracy: 0.3785 - val_loss: 1.7888
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3457 - loss: 1.7021 - val_accuracy: 0.4097 - val_loss: 1.7089
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3626 - loss: 1.7048 - val_accuracy: 0.4340 - val_loss: 1.6317
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4066 - loss: 1.6312 - val_accuracy: 0.4444 - val_loss: 1.5629
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3820 - loss: 1.6449 - val_accuracy: 0.4653 -

In [ ]:
# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 364, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
input_shape = (13,)

In [ ]:
# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization
for i in range(num_layers):
    model.add(layers.Dense(units[i]))
    model.add(layers.BatchNormalization())  # Add batch normalization
    model.add(layers.Activation(activations[i]))  # Apply activation after batch normalization
    model.add(layers.Dropout(dropouts[i]))  # Add dropout for regularization

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_46 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_10 (Activation)           │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_39 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_11 (Activation)           │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_48 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_12 (Activation)           │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ (None, 364)                 │         132,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_13 (Activation)           │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_42 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_14 (Activation)           │ (None, 52)                  │              

 Total params: 269,758 (1.03 MB)

 Trainable params: 267,548 (1.02 MB)

 Non-trainable params: 2,210 (8.63 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=500,batch_size=32, validation_data=(x_test, y_test_sparse))

Epoch 1/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.1747 - loss: 2.2541 - val_accuracy: 0.2535 - val_loss: 1.9670
Epoch 2/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.2976 - loss: 1.8221 - val_accuracy: 0.3403 - val_loss: 1.8610
Epoch 3/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3448 - loss: 1.7558 - val_accuracy: 0.3750 - val_loss: 1.7578
Epoch 4/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3726 - loss: 1.6770 - val_accuracy: 0.4062 - val_loss: 1.6736
Epoch 5/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4088 - loss: 1.5719 - val_accuracy: 0.4410 - val_loss: 1.5573
Epoch 6/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4397 - loss: 1.5221 - val_accuracy: 0.4618 - val_loss: 1.5127
Epoch 7/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4370 - loss: 1.5061 - val_accuracy: 0.4722 - val_loss: 1.4824
Epoch 8/500
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3898 - loss: 1.5321 - val_accuracy: 0.48

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=269,batch_size=64, validation_data=(x_test, y_test_sparse))

Epoch 1/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.8950 - loss: 0.3256 - val_accuracy: 0.6493 - val_loss: 1.2091
Epoch 2/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9309 - loss: 0.2099 - val_accuracy: 0.6319 - val_loss: 1.1901
Epoch 3/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9312 - loss: 0.2075 - val_accuracy: 0.6424 - val_loss: 1.1836
Epoch 4/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9162 - loss: 0.2569 - val_accuracy: 0.6458 - val_loss: 1.1596
Epoch 5/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9328 - loss: 0.1954 - val_accuracy: 0.6493 - val_loss: 1.1629
Epoch 6/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9211 - loss: 0.2385 - val_accuracy: 0.6424 - val_loss: 1.1660
Epoch 7/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9242 - loss: 0.2396 - val_accuracy: 0.6424 - val_loss: 1.1666
Epoch 8/269
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9492 - loss: 0.1791 - val_accuracy: 0.

In [ ]:
from tensorflow.keras.regularizers import l2

# Define the MLP model parameters
num_layers = 5
units = [65, 260, 364, 52, 52]
activations = ['relu', 'relu', 'relu', 'relu', 'relu']
dropouts = [0.1, 0.3, 0.4, 0.1, 0.1]
learning_rate = 0.0010343990168720423
loss_function = 'sparse_categorical_crossentropy'
l2_lambda = 0.01  # L2 regularization parameter
input_shape = (13,)  # Adjust based on your input feature size

# Build the MLP model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=input_shape))

# Hidden layers with batch normalization, dropout, and L2 regularization
for i in range(num_layers):
    model.add(layers.Dense(units[i],
                           kernel_regularizer=l2(l2_lambda)))  # L2 regularization added
    model.add(layers.BatchNormalization())  # Batch normalization
    model.add(layers.Activation(activations[i]))  # Activation
    model.add(layers.Dropout(dropouts[i]))  # Dropout

# Output layer (adjust the number of units to your number of classes)
model.add(layers.Dense(8, activation='softmax'))  # Assuming 8 classes for output

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                     │ (None, 65)                  │             910 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 65)                  │             260 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_10 (Activation)           │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 65)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 260)                 │          17,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 260)                 │           1,040 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_11 (Activation)           │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 260)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 364)                 │          95,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 364)                 │           1,456 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_12 (Activation)           │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 364)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 52)                  │          18,980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_13 (Activation)           │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 52)                  │           2,756 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 52)                  │             208 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_14 (Activation)           │ (None, 52)                  │              

 Total params: 138,406 (540.65 KB)

 Trainable params: 136,820 (534.45 KB)

 Non-trainable params: 1,586 (6.20 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=64, validation_data=(x_test, y_test_sparse))

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 115ms/step - accuracy: 0.1282 - loss: 7.9778 - val_accuracy: 0.2639 - val_loss: 7.3863
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2038 - loss: 7.2871 - val_accuracy: 0.2951 - val_loss: 6.9897
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2790 - loss: 6.7785 - val_accuracy: 0.3021 - val_loss: 6.6031
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2934 - loss: 6.3301 - val_accuracy: 0.3403 - val_loss: 6.2198
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3187 - loss: 5.9154 - val_accuracy: 0.3194 - val_loss: 5.8772
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3751 - loss: 5.4824 - val_accuracy: 0.2535 - val_loss: 5.5477
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3738 - loss: 5.1793 - val_accuracy: 0.2569 - val_loss: 5.2429
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3890 - loss: 4.8092 - val_accuracy: 0.2500 

### 4th

In [ ]:
# Define the HyperModel class:1
class MLPHyperModel7(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 12)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=130, max_value=390, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel7(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

Reloading Tuner from my_dir/mlp_tuning/tuner0.json


In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=64, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")

Best Hyperparameters:
num_layers: 3
units_0: 364
activation_0: relu
dropout_0: 0.4
learning_rate: 0.006019793254027166
loss: sparse_categorical_crossentropy
units_1: 377
activation_1: elu
dropout_1: 0.1
units_2: 91
activation_2: relu
dropout_2: 0.1
units_3: 221
activation_3: relu
dropout_3: 0.5
units_4: 286
activation_4: tanh
dropout_4: 0.5
units_5: 286
activation_5: softmax
dropout_5: 0.1
units_6: 156
activation_6: elu
dropout_6: 0.4


### 5th

In [ ]:
# Define the HyperModel class:1
class MLPHyperModel11(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 7)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=130, max_value=780, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel11(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

Reloading Tuner from my_dir/mlp_tuning/tuner0.json


In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=64, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")


Best Hyperparameters:
num_layers: 3
units_0: 364
activation_0: relu
dropout_0: 0.4
learning_rate: 0.006019793254027166
loss: sparse_categorical_crossentropy
units_1: 377
activation_1: elu
dropout_1: 0.1
units_2: 91
activation_2: relu
dropout_2: 0.1
units_3: 221
activation_3: relu
dropout_3: 0.5
units_4: 286
activation_4: tanh
dropout_4: 0.5
units_5: 286
activation_5: softmax
dropout_5: 0.1
units_6: 156
activation_6: elu
dropout_6: 0.4


### 6th

In [ ]:
# Define the HyperModel class:1
class MLPHyperModel12(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 7)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=130, max_value=1170, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model

In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel12(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=64, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

Trial 3 Complete [00h 00m 20s]
val_accuracy: 0.6076388955116272

Best val_accuracy So Far: 0.7013888955116272
Total elapsed time: 00h 01m 40s


In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")

Best Hyperparameters:
num_layers: 6
units_0: 819
activation_0: elu
dropout_0: 0.4
learning_rate: 0.0018936190094949422
loss: sparse_categorical_crossentropy
units_1: 130
activation_1: relu
dropout_1: 0.1
units_2: 130
activation_2: relu
dropout_2: 0.1
units_3: 130
activation_3: relu
dropout_3: 0.1
units_4: 130
activation_4: relu
dropout_4: 0.1
units_5: 130
activation_5: relu
dropout_5: 0.1


In [ ]:
# Define the hyperparameters
num_layers = 6
units_0 = 819
activation_0 = 'elu'
dropout_0 = 0.4
learning_rate = 0.0018936190094949422
loss_function = 'sparse_categorical_crossentropy'

# Units and activations for subsequent layers
layer_config = [
    {'units': 130, 'activation': 'relu', 'dropout': 0.1},
    {'units': 130, 'activation': 'relu', 'dropout': 0.1},
    {'units': 130, 'activation': 'relu', 'dropout': 0.1},
    {'units': 130, 'activation': 'relu', 'dropout': 0.1},
    {'units': 130, 'activation': 'relu', 'dropout': 0.1}
]

# Initialize the Sequential model
model = Sequential()

# Input layer with Batch Normalization
model.add(Dense(units_0, activation=activation_0, input_shape=(13,)))
model.add(BatchNormalization())
model.add(Dropout(dropout_0))

# Adding subsequent layers
for layer in layer_config:
    model.add(Dense(layer['units'], activation=layer['activation']))
    model.add(BatchNormalization())
    model.add(Dropout(layer['dropout']))

# Output layer (adjust number of output units to your problem)
model.add(Dense(8, activation='softmax'))  # 8 classes as an example for multiclass classification

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 819)                 │          11,466 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 819)                 │           3,276 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 819)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 130)                 │         106,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 130)                 │             520 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 130)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 130)                 │          17,030 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 130)                 │             520 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 130)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 130)                 │          17,030 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 130)                 │             520 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 130)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 130)                 │          17,030 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 130)                 │             520 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 130)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 130)                 │          17,030 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 130)                 │             520 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 130)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 193,110 (754.34 KB)

 Trainable params: 190,172 (742.86 KB)

 Non-trainable params: 2,938 (11.48 KB)

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=64, validation_data=(x_test, y_test_sparse))

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.1500 - loss: 2.6148 - val_accuracy: 0.2674 - val_loss: 1.9960
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2965 - loss: 1.9323 - val_accuracy: 0.2951 - val_loss: 1.9552
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3795 - loss: 1.7152 - val_accuracy: 0.2882 - val_loss: 1.8817
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4007 - loss: 1.6320 - val_accuracy: 0.3264 - val_loss: 1.8368
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4590 - loss: 1.5052 - val_accuracy: 0.3507 - val_loss: 1.8048
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4822 - loss: 1.3899 - val_accuracy: 0.3368 - val_loss: 1.7606
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4989 - loss: 1.3523 - val_accuracy: 0.3472 - val_loss: 1.7352
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5088 - loss: 1.3076 - val_accuracy: 0.

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=200,batch_size=100, validation_data=(x_test, y_test_sparse))

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9649 - loss: 0.1011 - val_accuracy: 0.6562 - val_loss: 1.5279
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9463 - loss: 0.1323 - val_accuracy: 0.6701 - val_loss: 1.4633
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9636 - loss: 0.1164 - val_accuracy: 0.6562 - val_loss: 1.4530
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9718 - loss: 0.0905 - val_accuracy: 0.6389 - val_loss: 1.5155
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9859 - loss: 0.0807 - val_accuracy: 0.6354 - val_loss: 1.5244
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9673 - loss: 0.0899 - val_accuracy: 0.6562 - val_loss: 1.4932
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9725 - loss: 0.0990 - val_accuracy: 0.6562 - val_loss: 1.5080
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9718 - loss: 0.0842 - val_accuracy: 0.

In [ ]:
hostory=model.fit(x_train, y_train_sparse, epochs=700,batch_size=16, validation_data=(x_test, y_test_sparse))

Epoch 1/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6630 - loss: 1.5160 - val_accuracy: 0.3715 - val_loss: 2.7161
Epoch 2/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4290 - loss: 1.6942 - val_accuracy: 0.4549 - val_loss: 1.6262
Epoch 3/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5170 - loss: 1.3621 - val_accuracy: 0.5069 - val_loss: 1.3467
Epoch 4/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5728 - loss: 1.1575 - val_accuracy: 0.5347 - val_loss: 1.2159
Epoch 5/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5940 - loss: 1.1170 - val_accuracy: 0.5833 - val_loss: 1.2490
Epoch 6/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6776 - loss: 0.9586 - val_accuracy: 0.5799 - val_loss: 1.2494
Epoch 7/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7048 - loss: 0.8819 - val_accuracy: 0.5799 - val_loss: 1.2134
Epoch 8/700
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6559 - loss: 0.9288 - val_accuracy: 0.5729 -

### 7th

In [ ]:
# Define the HyperModel class:1
class MLPHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1, 6)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=130, max_value=780, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.9, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model


In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=32, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

Trial 3 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.5972222089767456
Total elapsed time: 00h 00m 58s


In [ ]:
# Retrieve the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
for hp_name in best_hyperparameters.values.keys():
    print(f"{hp_name}: {best_hyperparameters.get(hp_name)}")

Best Hyperparameters:
num_layers: 2
units_0: 741
activation_0: tanh
dropout_0: 0.2
learning_rate: 0.09458731722205639
loss: sparse_categorical_crossentropy
units_1: 130
activation_1: relu
dropout_1: 0.1


###8th

In [ ]:
class MLPHyperModel12(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Number of hidden layers
        for i in range(hp.Int('num_layers', 1,9)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=130, max_value=1170, step=13),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu','tanh','softmax']),
                input_dim=13 if i == 0 else None
            ))
            model.add(BatchNormalization())
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

        # Output layer
        model.add(Dense(8, activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')),
            loss=hp.Choice('loss', ['sparse_categorical_crossentropy', 'categorical_crossentropy']),
            metrics=['accuracy']
        )
        return model

In [ ]:
# Initialize the tuner
tuner = RandomSearch(
    MLPHyperModel(),
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mlp_tuning'
)

Reloading Tuner from my_dir/mlp_tuning/tuner0.json


In [ ]:
tuner.search(x_train, y_train_sparse, epochs=100,batch_size=32, validation_data=(x_test, y_test_sparse))
best_model = tuner.get_best_models(num_models=1)[0]

### RNN

In [ ]:
# Initialize the model
model = Sequential()

# Simple RNN Layer
model.add(SimpleRNN(64, input_shape=(13, 1), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Another Simple RNN Layer
model.add(SimpleRNN(64))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fully connected (Dense) layer
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Output layer
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train_sparse, batch_size=50, epochs=100, validation_data=(x_test, y_test_sparse))

Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 304ms/step - accuracy: 0.1323 - loss: 2.9759 - val_accuracy: 0.2812 - val_loss: 1.9685
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1984 - loss: 2.5575 - val_accuracy: 0.2986 - val_loss: 1.8695
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2245 - loss: 2.3824 - val_accuracy: 0.3438 - val_loss: 1.7881
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2493 - loss: 2.2464 - val_accuracy: 0.3750 - val_loss: 1.7273
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2284 - loss: 2.2397 - val_accuracy: 0.4062 - val_loss: 1.6878
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2761 - loss: 2.1383 - val_accuracy: 0.4167 - val_loss: 1.6559
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2370 - loss: 2.1390 - val_accuracy: 0.4375 - val_loss: 1.5878
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2911 - loss: 1.9809 - val_accuracy: 0.4